### Imports

In [1]:
import os
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.initializers import Constant
import keras_metrics

import pickle

Using TensorFlow backend.


### Globals

In [2]:
BASE_DIR            = ''
TEXT_DATA_DIR       = os.path.join(BASE_DIR, 'data')
MAX_SEQUENCE_LENGTH = 200
MAX_NUM_WORDS       = 20000
TEST_SPLIT          = 0.1
GLOVE_DIR           = TEXT_DATA_DIR
EMBEDDING_DIM       = 100

### Data load

In [3]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            fpath = os.path.join(path, fname)
            if sys.version_info < (3,):
                f = open(fpath)
            else:
                f = open(fpath, encoding='latin-1')
            t = f.read()
            i = t.find('\n\n')  # skip header
            if 0 < i:
                t = t[i:]
            texts.append(t)
            f.close()
            labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 10000 texts.


### Tokenization

In [4]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 11336 unique tokens.
Shape of data tensor: (10000, 200)
Shape of label tensor: (10000, 2)


### Split the data set

In [5]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(TEST_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

### Prepare the embedding layer

In [6]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [7]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [8]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Prepare the model

In [9]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(256, 3, activation='relu')(embedded_sequences)
x = MaxPooling1D()(x)
x = Dropout(0.2)(x)
x = Conv1D(256, 3, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Dropout(0.2)(x)
x = Conv1D(256, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc', keras_metrics.precision(), keras_metrics.recall()])

model.fit(x_train, y_train,
          batch_size=256,
          epochs=20,
          validation_split=0.1)

Train on 8100 samples, validate on 900 samples
Epoch 1/20
8100/8100 [==============================] - 6s 686us/step - loss: 0.7915 - acc: 0.5586 - precision: 0.5567 - recall: 0.5826 - val_loss: 0.6445 - val_acc: 0.6978 - val_precision: 0.6367 - val_recall: 0.9056

In [10]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(x_val, y_val)

print('test_acc:', test_accuracy)
print('test_precision:', test_precision)
print('test_recall:', test_recall)

1000/1000 [==============================] - 1s 574us/step
test_acc: 0.919
test_precision: 0.9282786883343691
test_recall: 0.907815631080598


### Export the model in a standardised format

In [11]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


### Load the unlabelled dataset

In [13]:
testFile = open('test/mixed_test_reviews.txt', encoding = 'utf8')
testFileContent = testFile.read()
testFileContent = testFileContent.splitlines()

### Tokenize the test data

In [14]:
testSequences = tokenizer.texts_to_sequences(testFileContent)
testData = pad_sequences(testSequences, maxlen=MAX_SEQUENCE_LENGTH)

### Classify the samples

In [15]:
testDataProb = model.predict(testData)

In [16]:
testDataClass = testDataProb.argmax(axis=-1)

In [17]:
testDataClass

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

### Output the results

In [18]:
with open('results.txt', 'w') as f:
    for item in testDataClass:
        f.write("%s\n" % item)